Скрипт для извлечения набора RNC-архивов из ROM'а с последующей разархивацией.

Извлечение архивов производится с помощью утилиты ppami.exe для amiga (http://aminet.net/package/util/pack/RNC_ProPack), которая исполняется на эмуляторе AmigaOS (https://github.com/cnvogelg/amitools).

In [ ]:
import struct
import subprocess

In [ ]:
cadEditorDir = "d:/DEV/MYGIT/CadEditor/" #test at CadEditor v3.5
jbDir = cadEditorDir + "settings_sega_jungle_book/"
romName    = cadEditorDir + "Jungle Book, The (U) [!].gen"

Извлечение RNC-архива из массива байт

In [ ]:
def extractRncData(rom, offset):
    if rom[offset : offset + 3] != "RNC":
        raise Exception("Not RNC archive")
    size, = struct.unpack(">I", rom[offset+8 : offset+12])
    return rom[offset : offset + size + 0x12]

Вызов анпакера из в текущей директории. Смена рабочей директории не поддерживаются из-за бага vamos

In [ ]:
def callAmigaRncPacker(workDir, fnameIn):
    #"work:"+fnameIn] 
    #"-V", "work:%s"%workDir
    p = subprocess.Popen(["vamos", "ppami.exe", "p", "d", fnameIn], stdout = subprocess.PIPE, shell = True)
    return p

In [ ]:
def callAmigaRncUnpacker(workDir, fnameIn):
    #"work:"+fnameIn] 
    #"-V", "work:%s"%workDir
    p = subprocess.Popen(["vamos", "ppami.exe", "u", "d", ".bin" , fnameIn], stdout = subprocess.PIPE, shell = True)
    return p

Поиск и извлечение всех RNC-архивов из ROM
---

In [ ]:
with open(romName, "rb") as f:
    buf = f.read()

In [ ]:
i = 0    
while True: 
    i = buf.find("RNC", i+1)
    if i == -1:
        break
    rncVersion, = struct.unpack(">B", buf[i+3])
    if rncVersion > 2:
        continue
    #todo check special version for RNC0, that not packed and not stored sizeUnpacked
    size, = struct.unpack(">I", buf[i+8 : i+12]) 
    sizeUnpacked, = struct.unpack(">I", buf[i+4 : i+8])
    if size > sizeUnpacked:
        continue
    #todo add checking packed size CRC
    print "Found RNC at address:", hex(i), "Size:",size,"/", sizeUnpacked
    rncData = extractRncData(buf, i)
    mapFname = "map_"+hex(i)[2:]+".rnc"
    with open(mapFname, "wb") as fo:
        fo.write(rncData)
    p = callAmigaRncUnpacker(jbDir, mapFname)
    print p.stdout.read()
    print "----------------------------------"

Парсинг конфигов
---

Дампим в эмуляторе палитру в файл, находим в ROM палитру и указатель на неё, попадаем на конфиги всех уровней игры.
Размер конфига 68 байт, карта тайлов нулевого уровня по адресу 0x1A17EC, выводим данные о конфигах уровней, необходимые для отрисовки карты
Адреса специфичные для Jungle Book

In [ ]:
startAddr = 0x1A17EC
configSize = 68

In [ ]:
for x in xrange(16):
    print "Config for level", x
    curConfigAddr = startAddr + x*configSize
    
    print hex(curConfigAddr)
    map1, = struct.unpack(">I", buf[curConfigAddr + 8 : curConfigAddr + 12])
    print "  Map:", hex(map1 & 0x7FFFFF)
    mapW, mapH = struct.unpack(">HH", buf[curConfigAddr + 30: curConfigAddr + 34])
    print "  Map size  :", mapW,"x",mapH
    layerB, = struct.unpack(">I", buf[curConfigAddr + 14 : curConfigAddr + 18])
    print "  LayerB    :", hex(layerB)
    palAddr, = struct.unpack(">I", buf[curConfigAddr + 18 : curConfigAddr + 22])
    print "  Pal       :", hex(palAddr)
    tileMap, = struct.unpack(">I", buf[curConfigAddr + 22 : curConfigAddr + 26])
    print "  Tilemap   :", hex(tileMap)
    enemy, = struct.unpack(">I", buf[curConfigAddr : curConfigAddr + 4])
    print "  Enemy       :", hex(enemy)
    